# Image Data

https://github.com/deep-learning-with-pytorch/dlwpt-code/tree/master/data

In [1]:
import imageio.v2 as imageio
import torch
import torchvision
import numpy as np

img_arr = imageio.imread(
    'https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-dog/bobby.jpg'
    )
img_arr.shape  # (width, height, channels)

(720, 1280, 3)

PyTorch requires the arrays to be $C\times H \times W$. Permuting axes doesn't copy data, but changes the size and stride of the `torch.Tensor`.

In [2]:
isinstance(img_arr, np.ndarray)  # True

img = torch.from_numpy(img_arr).permute([2, 0, 1])
print(img.shape)

torch.Size([3, 720, 1280])


We will build a trivial batch of images, containing just our single image.

In [3]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [9]:
cats_uri_prefix = "https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/image-cats/"
for idx, filename in enumerate(["cat1.png", "cat2.png", "cat3.png"]):
    batch[idx] = torch.from_numpy(imageio.imread(cats_uri_prefix + filename)).permute([2,0,1])[:3]

In [11]:
batch.shape

torch.Size([3, 3, 256, 256])

# Tabular data

PyTorch tensors are homogeneous!

In [36]:
import pandas as pd
uri = "https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/tabular-wine/winequality-white.csv"

wineq_numpy = pd.read_csv(uri, delimiter=';')
col_list = list(wineq_numpy.columns)
wineq_numpy = wineq_numpy.to_numpy(dtype='float32')

In [37]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [47]:
data = wineq[:, :-1]
target = wineq[:, -1].long()  # integer labels

In [52]:
# one-hot encoding of labels
# integer encoding induces an ordering on the scores; one-hot does not
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)  #  in-place
# scatter(
#   dimension/axis along which the other two args are specified,
#   column tensor: indices to scatter (unsqueeze makes it a column tensor)
#   elements to scatter or a single scalar to scatter
# )

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

What does `scatter_` do?

Let's manipulate the rest of the data

In [58]:
data_mean = torch.mean(data, dim=0)  # get mean of columns
data_var = torch.var(data, dim=0)

data_normalized = (data - data_mean) / torch.sqrt(data_var)  # uses broadcasting
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7997e-02,  ...,  7.3995e-01,
          1.3417e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [59]:
target <= 3

tensor([False, False, False,  ..., False, False, False])

# Time series data

In [60]:
uri = "https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/bike-sharing-dataset/hour-fixed.csv"
bikes_numpy = np.loadtxt(
    uri,
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1: lambda x: float(x[8:10])})
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])